In [1]:
import itertools
import os
from pathlib import Path
import pickle
import sys

from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

sys.path.append('../../src')
from geoai_retail.analysis import get_add_new_closest_dataframe

sys.path.append('../')
import config

%load_ext autoreload
%autoreload 2

In [2]:
data = Path(os.path.abspath('../../data'))
interim_dir = data/'interim'
interim_gdb = interim_dir/'interim.gdb'
raw_dir = data/'raw'
raw_gdb = raw_dir/'raw.gdb'

origin_fc = str(raw_gdb/'sea_block_group')
origin_id_fld = 'ID'

dest_fc = str(raw_gdb/'sea_ace')
dest_id_fld = 'LOCNUM'

comp_fc = raw_gdb/'sea_ace_comp'
comp_id_fld = 'LOCNUM'

real_estate_fc = raw_gdb/'real_estate_hex'
real_estate_id_fld = 'GRID_ID'

closest_brand = interim_dir/'closest_store.csv'
closest_comp = interim_dir/'closest_competition.csv'
origin_demographics = interim_dir/'origin_enrich_all.csv'
inrix_trips = interim_dir/'cust_count_inrix.csv'
training_data = interim_dir/'training_data.csv'

In [3]:
origin_df = GeoAccessor.from_featureclass(str(origin_fc))
origin_df.head()

,OBJECTID,ID,NAME,SHAPE
0,1,530530701003,530530701.003,"{""rings"": [[[-122.06627999999995, 47.076520000..."
1,2,530530714071,530530714.071,"{""rings"": [[[-122.34031999999996, 47.071510000..."
2,3,530530714072,530530714.072,"{""rings"": [[[-122.35767999999996, 47.067370000..."
3,4,530530714073,530530714.073,"{""rings"": [[[-122.36847999999998, 47.067630000..."
4,5,530530714112,530530714.112,"{""rings"": [[[-122.41108999999994, 47.071690000..."


In [4]:
xgr = pickle.load(open('../../models/sea_ace_xgr_stage01.pkl', 'rb'))
xgr

[21:53:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [5]:
dest_df = GeoAccessor.from_featureclass(str(dest_fc))

In [6]:
master_df = pd.read_csv(training_data)
master_df[origin_id_fld] = master_df[origin_id_fld].astype('int64')

In [7]:
str_cols = master_df.select_dtypes('object').columns

factor_col_lst = [c for c in master_df.columns if
                  not c.startswith('destination_id') and
                  not c == origin_id_fld and
                  not c.endswith('_FY') and
                  not c.startswith('trip_count') and
                  not c in str_cols]
label_col = 'trip_count_01'

In [8]:
from sklearn.preprocessing import StandardScaler
std_sclr = StandardScaler()

In [9]:
ent_gis = GIS(config.ent_url, username=config.ent_user, password=config.ent_pass)
ent_gis

GIS @ https://geoai-webgis-ent.bd.esri.com/portal version:7.1

In [10]:
def get_new_tuple(row, real_estate_id_fld):
    new_location_id = int(row[real_estate_id_fld])
    cnt_coords = row['SHAPE'].centroid
    cnt_geom = Geometry({'x': cnt_coords[0], 'y': cnt_coords[1], 'spatialReference': real_estate_df.spatial.sr})
    return cnt_geom, new_location_id

def get_affected_df(input_tuple):
    
    cnt_geom, new_location_id = input_tuple
    
    # get a dataframe of all potential rebalanced areas
    rebal_df = get_add_new_closest_dataframe(
        origins=origin_df,
        origin_id_field=origin_id_fld,
        destinations=dest_df,
        destination_id_field=dest_id_fld,
        closest_table=closest_brand,
        new_destination=cnt_geom,
        gis=ent_gis
    )
    
    # while we soved way broader than possilby needed, we check and only keep changed rows
    dest_id_cols = [col for col in rebal_df.columns if col.startswith('destination_id')]
    dest_id_arr = dest_df[dest_id_fld].unique()
    new_rebal_origin_ids = pd.concat([rebal_df[~rebal_df[dest_col].isin(dest_id_arr)]['origin_id'] for dest_col in dest_id_cols]).unique()
    rebal_df = rebal_df[rebal_df['origin_id'].isin(new_rebal_origin_ids)]

    # cast the origin id to integer to avoid data type issues
    rebal_df['origin_id'] = rebal_df['origin_id'].astype('int64')

    # get the affected areas in the master dataframe, the one with all the factors
    affected_master = master_df[master_df['ID'].isin(rebal_df['origin_id'].unique())].copy()

    # if the columns are not in the new rebalanced dataframe, create a dataframe of values to keep
    keep_cols = [col for col in master_df.columns if not col in rebal_df]
    affected_keep = affected_master[keep_cols].copy()
    
    # if adding a new location does not affect any of the block groups, just return none
    if len(affected_master.index) == 0:
        return pd.DataFrame([[new_location_id, 0.0]], columns=['new_destination_id', 'delta_01'])
    
    else:

        # now, combine the factors data with the rebalanced data
        affected_df = affected_keep.join(rebal_df.set_index('origin_id'), on='ID')

        # infer new values for these changed areas
        X = std_sclr.fit_transform(affected_df[factor_col_lst])
        affected_df['predict_new_01'] = xgr.predict(X)

        # now, if any values are below zero, zero them out
        affected_df['predict_new_01'] = affected_df['predict_new_01'].apply(lambda val: 0.0 if val < 0 else val)

        # calcuate a change column
        affected_df['delta_01'] = affected_df['predict_new_01'] - affected_df['trip_count_01']

        # rename the id column to be somwhat more consistent
        affected_df.insert(0, 'origin_id', affected_df[origin_id_fld])
        affected_df.drop('ID', axis=1, inplace=True)

        # also, put the new destination unique identifier in, so results can be filtered
        affected_df.insert(1, 'new_destination_id', int(new_location_id))

        # trim down the output dataframe to just a few columns
        out_cols = [col for col in affected_df.columns 
                    if '_id' in col or
                    col.startswith('proxmity_') or
                    col.startswith('predict_') or 
                    col.startswith('delta_') or
                    col.startswith('trip_count')
                   ]
        out_df = affected_df[out_cols].copy()

        return out_df

In [11]:
real_estate_df = GeoAccessor.from_featureclass(str(real_estate_fc), where_clause="road_reachable = 1")
real_estate_df = real_estate_df[[real_estate_id_fld, 'SHAPE']].copy()
real_estate_df = real_estate_df.iloc[int(len(real_estate_df.index) * 0.75):]
print(real_estate_df.GRID_ID.dtype)
print(len(real_estate_df.index))
real_estate_df.head()

object
477


,GRID_ID,SHAPE
1429,3053,"{""rings"": [[[-121.46326378499998, 47.864097557..."
1430,3054,"{""rings"": [[[-121.43240685499995, 47.876048282..."
1431,3058,"{""rings"": [[[-121.30897913399997, 47.876048282..."
1432,3059,"{""rings"": [[[-121.27812220399994, 47.864097557..."
1433,3089,"{""rings"": [[[-122.32725782999995, 47.887996251..."


In [12]:
import warnings
warnings.filterwarnings('ignore')

real_estate_df.reset_index(inplace=True, drop=True)

for idx, row in real_estate_df.iterrows():
    rebal_result = get_affected_df(get_new_tuple(row, real_estate_id_fld))
    if idx == 0:
        affected_df = rebal_result
    else:
        affected_df = affected_df.append(rebal_result)
    print(f'success - {row[0]}')
    
    affected_df.to_csv(interim_dir/'ace_affected_04.csv')

Location "421027779" in "Facilities" is on a soft-restricted network element.

Location "530330226051" in "Incidents" is on a soft-restricted network element.
Location "530330322072" in "Incidents" is on a soft-restricted network element.
Location "530330322082" in "Incidents" is on a soft-restricted network element.
Location "530330322083" in "Incidents" is on a soft-restricted network element.
Location "530330322152" in "Incidents" is on a soft-restricted network element.
Location "530330323164" in "Incidents" is on a soft-restricted network element.
Location "530330323171" in "Incidents" is on a soft-restricted network element.
Location "530330323174" in "Incidents" is on a soft-restricted network element.
Location "530330323261" in "Incidents" is on a soft-restricted network element.
Location "530330323281" in "Incidents" is on a soft-restricted network element.
Location "530330323291" in "Incidents" is on a soft-restricted network element.
Location "421027779" in "Facilities" is o

success - 3053


Location "421027779" in "Facilities" is on a soft-restricted network element.

Location "530330226051" in "Incidents" is on a soft-restricted network element.
Location "530330322072" in "Incidents" is on a soft-restricted network element.
Location "530330322082" in "Incidents" is on a soft-restricted network element.
Location "530330322083" in "Incidents" is on a soft-restricted network element.
Location "530330322152" in "Incidents" is on a soft-restricted network element.
Location "530330323164" in "Incidents" is on a soft-restricted network element.
Location "530330323171" in "Incidents" is on a soft-restricted network element.
Location "530330323174" in "Incidents" is on a soft-restricted network element.
Location "530330323261" in "Incidents" is on a soft-restricted network element.
Location "530330323281" in "Incidents" is on a soft-restricted network element.
Location "530330323291" in "Incidents" is on a soft-restricted network element.
Location "421027779" in "Facilities" is o

success - 3054


Location "421027779" in "Facilities" is on a soft-restricted network element.

Location "530330226051" in "Incidents" is on a soft-restricted network element.
Location "530330322072" in "Incidents" is on a soft-restricted network element.
Location "530330322082" in "Incidents" is on a soft-restricted network element.
Location "530330322083" in "Incidents" is on a soft-restricted network element.
Location "530330322152" in "Incidents" is on a soft-restricted network element.
Location "530330323164" in "Incidents" is on a soft-restricted network element.
Location "530330323171" in "Incidents" is on a soft-restricted network element.
Location "530330323174" in "Incidents" is on a soft-restricted network element.
Location "530330323261" in "Incidents" is on a soft-restricted network element.
Location "530330323281" in "Incidents" is on a soft-restricted network element.
Location "530330323291" in "Incidents" is on a soft-restricted network element.
Location "421027779" in "Facilities" is o

...

success - 3294


Location "421027779" in "Facilities" is on a soft-restricted network element.

Location "421027779" in "Facilities" is on a soft-restricted network element.
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Gates").

Location "530610523011" in "Incidents" is on a soft-restricted network element.
Location "421027779" in "Facilities" is on a soft-restricted network element.
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Private Roads" "Avoid Gates").

Location "530610535052" in "Incidents" is on a soft-restricted network element.
Location "530610536021" in "Incidents" is on a soft-restricted network element.
Location "421027779" in "Facilities" is on a soft-restricted network element.
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Private Roads" "Avoid Gates").

Location "421027779" in "Facilities" is on a soft-restr

success - 3295


Location "421027779" in "Facilities" is on a soft-restricted network element.



TypeError: string indices must be integers